In [1]:
# Ignore cuDDa warning messages
import warnings
warnings.filterwarnings('ignore')

# Enable GPU
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# # Expands the Jupyter Notebook Output Size to fit your window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Load in tensorboard
%load_ext tensorboard

from tensorflow_models import TF_Models, Ein_Multiply, leaky_relu, rank_loss_func
from graph_predictions import Graph_Predictions

# tf.config.list_physical_devices()

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# # Update the directory since all files were moved to "ignorable_data"
# import os
# path = os.getcwd()
# os.chdir(path + '\ignorable_data')

In [ ]:
# tf.config.run_functions_eagerly(True)

alpha = [1, 1e-1, 0, 1e-2, 1e1, 1e2]
alpha = [1]
beta = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]
# alpha.sort(reverse=True)

model_folder = './ignorable_data/alpha_1000_beta_0.01/models'
predictions_folder = './ignorable_data/prediction_results/'
data_block_folder = './ignorable_data/alpha_1000_beta_0.01/data_blocks/'

DMJ = TF_Models('./ignorable_data/data_sets/NASDAQ_Cleaned', model_folder, reload=False)
data_splits = DMJ.split_data()
# model = DMJ.generate_model()
# GP = Graph_Predictions(model_folder, "./strategies/RL_validation_strategies", DMJ)

# MSE TEST

for a in alpha:
    for b in beta:
        
        if not (a == 1 and b == 1):
            if a == b:
                continue
                
        # Reset the training object to get rid of old data
        DMJ = TF_Models('./ignorable_data/data_sets/NASDAQ_Cleaned', model_folder, reload=False)
        
        # Create the model using parameters we're tuning
        DMJ._generate_model(model_type='lstm', loss_function='mse', activation='leaky_relu', hidden_units=64, true_random=True, alpha=a, beta=b)
        
        # Have it train as much as it can
        DMJ.train_model_loop(epoch_batches=15)
        
        # Save the model with a tag
#         DMJ.save_model(tag=f'E2E_LSTM_ValSet_{a}-ALPHA{b}-BETA_SD100998_expanding_window')
        DMJ.save_model(tag=f'E2E_LSTM_ValSet_TFMSE_SD100998_expanding_window')
        
        # Reset the training object to get rid of old data
        GP = Graph_Predictions(model_folder, "./ignorable_data/strategies/RL_validation_strategies", DMJ)
        
        # Generate the prediction file
        GP.generate_prediction_json(DMJ.model_name, predictions_folder, neural_net_type='lstm')
        
        # Create the diagnostics file for the most recently saved model
        GP.generate_model_diagnostics(predictions_folder + GP.model_name, datablock_folder='./ignorable_data/alpha_1000_beta_0.01/data_blocks/')
        
        input('Wait ')

In [7]:
model_folder = './ignorable_data/alpha_1000_beta_0.01/models'
model_folder = './ignorable_data/models/'
model_name = '01-04-2021--15--57--SEP_LSTM_GCN3-5e-6LR--820Epochs--mse-Loss--64-HU--'

new_name = '01-04-2021--15--57--SEP_LSTM_GCN3-5e-6LR--820Epochs--mse-Loss--64-HU--400_slidewindow'

directory = "./ignorable_data/alpha_1000_beta_0.01/models/"
predictions_folder = './ignorable_data/alpha_1000_beta_0.01/models/'

DMJ = TF_Models('./ignorable_data/data_sets/NASDAQ_Cleaned', model_folder, reload=False)

# For training over sliding window
# Reset the training object to get rid of old data
GP = Graph_Predictions(model_folder, "./ignorable_data/strategies/RL_validation_strategies", DMJ)

# Generate the prediction file
GP.generate_prediction_json(model_name, directory, neural_net_type='gcn', name_override=new_name, sliding_window=800)

# Create the diagnostics file for the most recently saved model
GP.generate_model_diagnostics(predictions_folder + GP.model_name, datablock_folder='./ignorable_data/alpha_1000_beta_0.01/data_blocks/')


Loading Model: '01-04-2021--15--57--SEP_LSTM_GCN3-5e-6LR--820Epochs--mse-Loss--64-HU--'
Total number of days: 309
Day 0 | Day 1 | Day 2 | Day 3 | Day 4 | Day 5 | Day 6 | Day 7 | Day 8 | Day 9 | Day 10 | Day 11 | Day 12 | Day 13 | Day 14 | Day 15 | Day 16 | Day 17 | Day 18 | Day 19 | Day 20 | Day 21 | Day 22 | Day 23 | Day 24 | Day 25 | Day 26 | Day 27 | Day 28 | Day 29 | Day 30 | Day 31 | Day 32 | Day 33 | Day 34 | Day 35 | Day 36 | Day 37 | Day 38 | Day 39 | Day 40 | Day 41 | Day 42 | Day 43 | Day 44 | Day 45 | Day 46 | Day 47 | Day 48 | Day 49 | Day 50 | Day 51 | Day 52 | Day 53 | Day 54 | Day 55 | Day 56 | Day 57 | Day 58 | Day 59 | Day 60 | Day 61 | Day 62 | Day 63 | Day 64 | Day 65 | Day 66 | Day 67 | Day 68 | Day 69 | Day 70 | Day 71 | Day 72 | Day 73 | Day 74 | Day 75 | Day 76 | Day 77 | Day 78 | Day 79 | Day 80 | Day 81 | Day 82 | Day 83 | Day 84 | Day 85 | Day 86 | Day 87 | Day 88 | Day 89 | Day 90 | Day 91 | Day 92 | Day 93 | Day 94 | Day 95 | Day 96 | Day 97 | Day 98 | Day 

In [ ]:
DMJ.history.history['val_loss']
DMJ.history.history['lr']
DMJ.epochs_n

In [ ]:
# Train the  model
model = DMJ.train_model(epochs=500)
# model = DMJ.train_model(model, data_splits['x_train'], data_splits['y_train'], data_splits['x_val'], data_splits['y_val'], epochs=50, learning_rate=5e-5, gcn_matrix=DMJ.Normalized_Adjacency_Matrix)

In [ ]:
DMJ.save_model(tag='E2E_LSTM_VariedLR_RL_ALPHA_1_NOMSE')

In [ ]:
xGP = Graph_Predictions("./models", "./strategies", DMJ)

In [ ]:
# GP.strategy_ratio_lstm('11-22-2020--17--32--LSTM--50Epochs--mse-Loss--64-HU--None', avoid_fall=False, name_override='test_')
# GP.strategy_ratio_lstm('11-22-2020--17--32--LSTM--50Epochs--mse-Loss--64-HU--None', avoid_fall=False, average=10, name_override='LSTM-MSE-50Epoch-Average10')
# GP.strategy_ratio_lstm('11-22-2020--17--32--LSTM--50Epochs--mse-Loss--64-HU--None', avoid_fall=True, name_override='LSTM-MSE-50Epoch-AvoidFall')

# GP.strategy_ratio_lstm('11-22-2020--14--55--LSTM-Rankloss--50Epochs--rlf-Loss--64-HU--', avoid_fall=False, name_override='LSTM-RankLoss-50Epoch')
# GP.strategy_ratio_lstm('11-22-2020--14--55--LSTM-Rankloss--50Epochs--rlf-Loss--64-HU--', avoid_fall=False, average=10, name_override='LSTM-RankLoss-50Epoch-Average10')
# GP.strategy_ratio_lstm('11-22-2020--14--55--LSTM-Rankloss--50Epochs--rlf-Loss--64-HU--', avoid_fall=True, name_override='LSTM-RankLoss-50Epoch-AvoidFall')

# GP.strategy_ratio_gcn(r'11-22-2020--16--31--GCN1-MSE--50Epochs--mse-Loss--64-HU--', avoid_fall=False, average=10, name_override='GCN1-MSE-50Epoch-Average10')
# GP.strategy_ratio_gcn(r'11-22-2020--16--31--GCN1-MSE--50Epochs--mse-Loss--64-HU--', avoid_fall=True, name_override='GCN1-MSE-50Epoch-AvoidFall')

# GP.strategy_ratio_gcn(r'11-22-2020--16--34--GCN1-Rankloss--50Epochs--rlf-Loss--64-HU--', avoid_fall=False, average=10, name_override='GCN1-RankLoss-50Epoch-Average10')
# GP.strategy_ratio_gcn(r'11-22-2020--16--34--GCN1-Rankloss--50Epochs--rlf-Loss--64-HU--', avoid_fall=True, name_override='GCN1-RankLoss-50Epoch-AvoidFall')

# GP.strategy_ratio_gcn(r'11-22-2020--16--37--GCN2-MSE--50Epochs--mse-Loss--64-HU--', avoid_fall=False, average=10, name_override='GCN2-MSE-50Epoch-Average10')
# GP.strategy_ratio_gcn(r'11-22-2020--16--37--GCN2-MSE--50Epochs--mse-Loss--64-HU--', avoid_fall=True, name_override='GCN2-MSE-50Epoch-AvoidFall')

# GP.strategy_ratio_gcn(r'11-22-2020--16--39--GCN2-RankLoss--50Epochs--rlf-Loss--64-HU--', avoid_fall=False, average=10, name_override='GCN2-RankLoss-50Epoch-Average10')
# GP.strategy_ratio_gcn(r'11-22-2020--16--39--GCN2-RankLoss--50Epochs--rlf-Loss--64-HU--', avoid_fall=True, name_override='GCN2-RankLoss-50Epoch-AvoidFall')

# GP.strategy_ratio_gcn('11-22-2020--16--46--GCN3-MSE--50Epochs--mse-Loss--64-HU--', avoid_fall=False, average=10, name_override='GCN3-MSE-50Epoch-Average10')
# GP.strategy_ratio_gcn(r'11-22-2020--16--46--GCN3-MSE--50Epochs--mse-Loss--64-HU--', avoid_fall=True, name_override='GCN3-MSE-50Epoch-AvoidFall-2ndBest')

# GP.strategy_ratio_gcn('11-22-2020--16--43--GCN3-RankLoss--50Epochs--rlf-Loss--64-HU--', avoid_fall=False, average=10, name_override='GCN3-RankLoss-50Epoch-Average10')
# GP.strategy_ratio_gcn(r'11-22-2020--16--43--GCN3-RankLoss--50Epochs--rlf-Loss--64-HU--', avoid_fall=True, name_override='GCN3-RankLoss-50Epoch-AvoidFall')

# GP.strategy_ratio_gcn('11-22-2020--20--46--GCN3-MSE-Ratio+1--50Epochs--mse-Loss--64-HU--', avoid_fall=True, average=1, name_override='GCN3-MSE-200Epoch-Ratio+1')
# GP.strategy_ratio_gcn('11-22-2020--20--43--GCN3-RankLoss-Ratio+1--50Epochs--rlf-Loss--64-HU--', avoid_fall=True, average=1, name_override='GCN3-RankLoss-200Epoch-Ratio+1')

'''New prediction memory storage'''
# GP.generate_prediction_json('11-22-2020--15--01--LSTM-MSE--50Epochs--mse-Loss--64-HU--', neural_net_type='lstm')
# GP.generate_prediction_json('11-22-2020--16--46--GCN3-MSE--50Epochs--mse-Loss--64-HU--', neural_net_type='gcn')

'''Seperately trained LSTM combo'''
# GP.generate_prediction_json('01-04-2021--14--20--SEP_LSTM_GCN3-1e-5LR--70Epochs--mse-Loss--64-HU--', neural_net_type='gcn')

''''''
GP.generate_prediction_json('02-04-2021--15--11--E2E_LSTM_VariedLR_RL_ALPHA_1_NOMSE--500Epochs--rlf-Loss--64-HU--', neural_net_type='lstm')

In [ ]:
strat_name = '02-04-2021--15--11--E2E_LSTM_VariedLR_RL_ALPHA_1_NOMSE--500Epochs--rlf-Loss--64-HU--_PM'

# Testing the PM file feature
avg = [1, 5, 20, 50, 100, 200]
avg = [1]
for a in avg:
#     GP.prediction_json_strategy_max_entities(strat_name, average=a, avoid_fall=False, name_override=strat_name+ f'{a}AVG')
    GP.prediction_json_strategy_determine_best(strat_name, average=a, avoid_fall=False, name_override=strat_name+ f'{a}AVG_Correct_BuyDay_plus1')
    GP.save_results()
# Testing the mse tracking feature
# GP.prediction_json_mse('01-04-2021--15--57--SEP_LSTM_GCN3-5e-6LR--820Epochs--mse-Loss--64-HU--_PM')
GP.save_results()

In [ ]:
GP.generate_model_diagnostics('02-04-2021--15--11--E2E_LSTM_VariedLR_RL_ALPHA_1_NOMSE--500Epochs--rlf-Loss--64-HU--_PM', datablock_folder='RL_validation_set')

'01-04-2021--15--57--SEP_LSTM_GCN3-5e-6LR--820Epochs--mse-Loss--64-HU--_PM'
255025890000.0
0.012249682697757544
0.7766990291262136


In [ ]:
# GP.generate_upper_lower_avg_bounds()

GP.display_graph()

In [ ]:
# GP.strategy_ratio_gcn('11-22-2020--16--46--GCN3-MSE--50Epochs--mse-Loss--64-HU--', avoid_fall=False, average=10, name_override='GCN3-MSE-50Epoch-Average10')
# GP.strategy_ratio_gcn(r'11-22-2020--16--46--GCN3-MSE--50Epochs--mse-Loss--64-HU--', avoid_fall=True, name_override='GCN3-MSE-50Epoch-AvoidFall')

# GP.strategy_ratio_gcn('11-22-2020--16--43--GCN3-RankLoss--50Epochs--rlf-Loss--64-HU--', avoid_fall=False, average=10, name_override='GCN3-RankLoss-50Epoch-Average10')
# GP.strategy_ratio_gcn(r'11-22-2020--16--43--GCN3-RankLoss--50Epochs--rlf-Loss--64-HU--', avoid_fall=True, name_override='GCN3-RankLoss-50Epoch-AvoidFall')

In [ ]:
# GP.save_results('./strategies')

In [ ]:
import ipywidgets as widgets
from ipywidgets import GridBox

In [ ]:
import time
bar = widgets.IntProgress(min=0, max=10, description='Loading:', bar_style='info')
display(bar)

for i in range(11):
    time.sleep(0.2)
    bar.value = i

In [ ]:

widgets.Text(
    value='Loading',
    description='',
    disabled=True
    layout=L
)



In [ ]:
# Start the loading bar by initializing it
nam_bar = widgets.IntProgress(min=0, max=5, value=0, description='Loading Normalized Adjacency Matrix:',
                              layout=widgets.Layout(width='auto'))
text = widgets.Text(value='Loading', description='', disabled=True, layout=widgets.Layout(width='auto'))

test = widgets.GridBox(children=[text, nam_bar], layout=widgets.Layout(width='auto'))

display(test)

In [ ]:
x_test = [1, 2, 3, 4, 5]

In [ ]:
x_test[0:1]

In [ ]:
for i in range(0, 10):
    print(i)

In [ ]:
%tensorboard --logdir logs/fit


In [ ]:
import numpy as np
np.amax(GP.rr_test[:, 0])

In [ ]:
np.argmax(GP.rr_test[:, 0])

In [ ]:
GP.rr_test[476,0]

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(GP.rr_test[5, :], GP.rr_test[100, :])

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

def swap_random(seq):
    idx = range(len(seq))
    i1, i2 = random.sample(idx, 2)
    seq[i1], seq[i2] = seq[i2], seq[i1]

trials_RL = []
for e in range(100,300):
    RL = []
    for t in range(300):
        A = []; B = [];
        for l in range(e):
            n = random.uniform(-1, 1)
            A.append(n)
            B.append(n)
        swap_random(B)
        swap_random(B)
        swap_random(B)
        swap_random(B)
        
        return_ratio = tf.constant(A, shape=(len(A), 1))
        ground_truth = tf.constant(B, shape=(len(B), 1))

        ###############################################################
        # Create an array of all_ones so that we can calculate all permutations of subtractions
        all_ones = tf.ones([len(return_ratio), 1], dtype=tf.float32)

        # Creates a N x N matrix with every predicted return ratio for each company subtracted with every other
        # company
        pred_dif = tf.math.subtract(
            tf.matmul(return_ratio, all_ones, transpose_b=True),
            tf.matmul(all_ones, return_ratio, transpose_b=True)
        )

        # Creates an N x N matrix containing every actual return ratio for each company subtracted with every other
        # company By switching the order of the all_ones matricies and the actual prices, a negative sign is introduced
        # When RELU is applied later, correct predictions will not affect loss while incorrect predictions will affect
        # loss depending on how incorrect the prediction was
        actual_dif = tf.math.subtract(
            tf.matmul(all_ones, ground_truth, transpose_b=True),
            tf.matmul(ground_truth, all_ones, transpose_b=True)
        )

        # Using the above two qualities, the algorithm can be punished for incorrectly calculating when a company is
        # doing better than another company Reduces the mean across each dimension until only 1 value remains
        rank_loss = tf.reduce_mean(
            # Takes if a given value is >0, it is kept, otherwise, it becomes 0
            tf.nn.relu(
                # Multiplies all of the
                tf.multiply(pred_dif, actual_dif)
            )
        )
        RL.append(rank_loss)
    trials_RL.append(np.mean(RL))

plt.plot(trials_RL)

In [ ]:
r = [0.1, 0.5, 0.2, 0.3]
print(list(zip(range(4), r)))

In [ ]:
predictions = [100, 300, 200, 400]
predictions = list(zip(range(len(predictions)), predictions))
predictions

In [ ]:
predictions.sort(key=lambda x: x[1], reverse=True)
predictions

In [ ]:
import numpy as np

# Normal Rank
A = list(range(1, 10001))
B = [i**(-1) for i in A]
print(np.mean(A))
print(np.mean(B)**-1)


'000_Avg_RR.p'
'000_Highest_RR_Possible.p'
'000_Lowest_RR_Possible.p'

In [ ]:
import similaritymeasures as sm
import numpy as np

# Generate random experimental data
n = 5
x = list(range(n))
y = [11, 26, 26, 11, -60]
exp_data = np.zeros((n, 2))
exp_data[:, 0] = x
exp_data[:, 1] = y

# Generate random numerical data
x = list(range(n))
y = [1, 2, 30, 4, -9000]
num_data = np.zeros((n, 2))
num_data[:, 0] = x
num_data[:, 1] = y

area = sm.area_between_two_curves(exp_data, num_data)
print(area)

In [ ]:
pm_name = 'uhhhh.json'
# If the .json file was already attached, this will fix the problem
pm_name = pm_name.split('.json')
pm_name = pm_name[0]

In [ ]:
print(pm_name)